# 0. Imports

In [1]:
from llmstudio import LLM
import os
import json
import dotenv
import requests

# Load environment variables from .env file
dotenv.load_dotenv()

True

# 1. Raw Vertex Call

In [2]:
model = 'gemini-1.5-flash'
api_key = os.getenv("GOOGLE_API_KEY")
url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:streamGenerateContent?alt=sse"
headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": api_key,
}

Running LLMstudio Engine on http://localhost:50928 
Running LLMstudio Tracking on http://localhost:50929 


In [3]:
calculator = [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]

calculator_2 = """function_declarations {
  name: "multiply"
  description: "Returns the product of two numbers."
  parameters {
    type_: OBJECT
    properties {
      key: "b"
      value {
        type_: NUMBER
      }
    }
    properties {
      key: "a"
      value {
        type_: NUMBER
      }
    }
    required: "a"
    required: "b"
  }
}"""

In [4]:
message = 'What is 9 times 10?'
data = {
    "system_instruction": {
        "parts": {
            "text": "You are a cat. Your name is Neko."
        }
    },
    "contents": [
        {
            "role": "user",
            "parts": [{"text": "Hello there"}]
        }
    ]
}

In [14]:
response = requests.post(url, headers=headers, json=vertexai_format, stream=False)

In [15]:
response.text

'data: {"candidates": [{"content": {"parts": [{"text": "Hello"}],"role": "model"},"finishReason": "STOP","index": 0}],"usageMetadata": {"promptTokenCount": 8,"candidatesTokenCount": 1,"totalTokenCount": 9}}\r\n\r\ndata: {"candidates": [{"content": {"parts": [{"text": "! ð\x9f\x91\x8b How can I help you today? ð\x9f\x98\x8a \\n"}],"role": "model"},"finishReason": "STOP","index": 0,"safetyRatings": [{"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_HATE_SPEECH","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_HARASSMENT","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_DANGEROUS_CONTENT","probability": "NEGLIGIBLE"}]}],"usageMetadata": {"promptTokenCount": 8,"candidatesTokenCount": 11,"totalTokenCount": 19}}\r\n\r\n'

# Vertex Functions Tests - Str to OAI to Vertex

In [13]:
def parse_chat_input(chat_input):
    # Check if the input is a simple string
    if isinstance(chat_input, str):
        # Parse the string into the OpenAI message format
        return [{"role": "user", "content": chat_input}]
    else:
        # Handle cases where the input is not a string (optional)
        raise ValueError("Input must be a string.")

# Example usage
input_string = "Hello, how can I help you?"
parsed_message = parse_chat_input(input_string)
print(parsed_message)


[{'role': 'user', 'content': 'Hello, how can I help you?'}]


In [13]:
def convert_openai_to_vertexai(messages):
    # Initialize the Vertex AI format
    vertexai_format = {
        "system_instruction": {
            "parts": {
                "text": ""
            }
        },
        "contents": []
    }
    
    # Loop through the OpenAI formatted messages
    for message in messages:
        if message["role"] == "system":
            # Set the system instruction
            vertexai_format["system_instruction"]["parts"]["text"] = message["content"]
        elif message["role"] == "user" or message["role"] == "assistant":
            # Convert roles: 'assistant' -> 'model'
            role = "model" if message["role"] == "assistant" else "user"
            # Append the message to the contents list in Vertex AI format
            vertexai_format["contents"].append({
                "role": role,
                "parts": [{"text": message["content"]}]
            })
    
    return vertexai_format

# Example usage
openai_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi there!"}
]

vertexai_format = convert_openai_to_vertexai(openai_messages)
print(json.dumps(vertexai_format, indent=2))


{
  "system_instruction": {
    "parts": {
      "text": "You are a helpful assistant."
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "Hello"
        }
      ]
    }
  ]
}


In [ ]:
[{"role": "system", "content": "You are a helpful assistant."},
 {"role": "user", "content": "Hello"},
 {"role": "assistant", "content": "Hi there!"},]